# Analyze Test Results and Generate Report (saga_pattern)

This notebook helps analyze test outputs from `saga_pattern` test runs, compute metrics, visualize results, and generate an English report file `saga_pattern/analyze_test_results.md`.

Sections:
1. Project setup: create directories and English-named report file
2. Load test result files (CSV / JSON / logs)
3. Parse and normalize filenames and encodings (rename non-ASCII to English)
4. Data cleaning and preprocessing
5. Compute key metrics and statistical summaries
6. Visualizations (time series, histograms, heatmaps)
7. Export results and write English markdown report (`saga_pattern/analyze_test_results.md`)
8. Unit tests for analysis functions (pytest)
9. Execute notebook programmatically and capture outputs


In [ ]:
# Section 1: Project setup - create directories and English-named report file
from pathlib import Path

saga_dir = Path.cwd()
report_path = saga_dir / 'analyze_test_results.md'

# Create report file if it doesn't exist
if not report_path.exists():
    report_path.write_text('# Test Results Analysis\n\nReport generated by analyze_test_results.ipynb\n')

print('Report path:', report_path)


In [ ]:
# Section 2: Load test result files (CSV / JSON / logs)
from pathlib import Path
import pandas as pd
import json

artifacts_dir = saga_dir / 'test_artifacts'
artifacts_dir.mkdir(exist_ok=True)

# Discover files (csv, json, txt, log)
file_patterns = ['*.csv', '*.json', '*.log', '*.txt']
files = []
for pat in file_patterns:
    files.extend(list(artifacts_dir.glob(pat)))

print('Found files:', files)

# Load CSV/JSON into DataFrame list
frames = []
for f in files:
    if f.suffix == '.csv':
        frames.append(pd.read_csv(f))
    elif f.suffix == '.json':
        try:
            df = pd.read_json(f)
        except ValueError:
            with open(f, 'r') as fh:
                data = json.load(fh)
            df = pd.json_normalize(data)
        frames.append(df)
    else:
        # Simple log parser: read lines into DataFrame
        with open(f, 'r', encoding='utf-8', errors='replace') as fh:
            lines = [l.strip() for l in fh if l.strip()]
        df = pd.DataFrame({'raw': lines})
        frames.append(df)

if frames:
    unified = pd.concat(frames, ignore_index=True, sort=False)
else:
    unified = pd.DataFrame()

print('Unified shape:', unified.shape)
unified.head()


In [ ]:
# Section 3: Parse and normalize filenames and encodings (rename non-ASCII to English)
import os

def normalize_name(p: Path) -> Path:
    # Simple mapping
    mapping = {
        'テスト結果を分析する.md': 'analyze_test_results.md'
    }
    if p.name in mapping:
        return p.with_name(mapping[p.name])
    # fallback: replace non-ascii with '_'
    name = ''.join((c if ord(c) < 128 else '_') for c in p.name)
    return p.with_name(name)

# Find files with non-ascii name in saga_dir
for p in saga_dir.iterdir():
    if any(ord(c) >= 128 for c in p.name):
        target = normalize_name(p)
        try:
            p.rename(target)
            print(f'Renamed {p.name} -> {target.name}')
        except Exception as e:
            print('Rename failed:', p, e)


In [ ]:
# Section 4: Data cleaning and preprocessing
import pandas as pd

# Example cleaning function

def clean_unified(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    # normalize column names
    df.columns = [str(c).strip().lower().replace(' ', '_') for c in df.columns]
    # parse timestamps if present
    for col in df.columns:
        if 'time' in col or 'timestamp' in col or 'date' in col:
            try:
                df[col] = pd.to_datetime(df[col], errors='coerce')
            except Exception:
                pass
    # numeric conversion
    for col in df.select_dtypes(include=['object']).columns:
        # try to convert common numeric columns
        if any(k in col for k in ('time', 'duration', 'ms', 'sec')):
            df[col] = pd.to_numeric(df[col], errors='coerce')
    # drop rows where everything is NaN
    df = df.dropna(how='all')
    return df

unified = clean_unified(unified)  # reuse unified from earlier cell
print('After cleaning shape:', unified.shape)
unified.head()


In [ ]:
# Section 5: Compute key metrics and statistical summaries
import numpy as np


def compute_metrics(df: pd.DataFrame) -> dict:
    metrics = {}
    total = len(df)
    metrics['total'] = total
    if 'status' in df.columns:
        passes = (df['status'].str.lower() == 'success').sum()
        metrics['passes'] = int(passes)
        metrics['pass_rate'] = passes / total if total else None
    # durations
    dur_cols = [c for c in df.columns if 'duration' in c or 'response_time' in c or 'ms' in c]
    if dur_cols:
        d = df[dur_cols].apply(pd.to_numeric, errors='coerce')
        metrics['duration_mean'] = d.mean(numeric_only=True).to_dict()
        metrics['duration_median'] = d.median(numeric_only=True).to_dict()
    # failure counts by scenario
    if 'scenario' in df.columns:
        metrics['failure_counts'] = df[df['status'].str.lower() != 'success']['scenario'].value_counts().to_dict()
    return metrics

metrics = compute_metrics(unified)
metrics


In [ ]:
# Section 6: Visualizations (time series, histograms, heatmaps)
import matplotlib.pyplot as plt
import seaborn as sns

# Time series of failures
if 'timestamp' in unified.columns or 'created_at' in unified.columns:
    time_col = 'timestamp' if 'timestamp' in unified.columns else 'created_at'
    df_time = unified.copy()
    df_time[time_col] = pd.to_datetime(df_time[time_col], errors='coerce')
    df_time = df_time.dropna(subset=[time_col])
    df_time['date'] = df_time[time_col].dt.floor('D')
    daily = df_time.groupby(['date', 'status']).size().unstack(fill_value=0)
    if not daily.empty:
        daily.plot(kind='line', figsize=(10,4))
        plt.title('Daily test counts by status')
        plt.show()

# Histogram of response_time
if 'response_time' in unified.columns:
    plt.figure(figsize=(8,4))
    sns.histplot(unified['response_time'].dropna(), kde=False)
    plt.title('Response Time Distribution')
    plt.xlabel('seconds')
    plt.show()

# Heatmap example: failures by scenario vs status (pivot)
if 'scenario' in unified.columns and 'status' in unified.columns:
    pivot = unified.pivot_table(index='scenario', columns='status', aggfunc='size', fill_value=0)
    if not pivot.empty:
        plt.figure(figsize=(8,6))
        sns.heatmap(pivot, annot=True, fmt='d', cmap='Reds')
        plt.title('Failures by Scenario and Status')
        plt.show()

# Save figures path
figs_dir = saga_dir / 'analysis_figs'
figs_dir.mkdir(exist_ok=True)
plt.savefig(figs_dir / 'last_plot.png')
print('Saved example fig to', figs_dir)


In [ ]:
# Section 7: Export results and write English markdown report (saga_pattern/analyze_test_results.md)
report_md = saga_dir / 'analyze_test_results.md'

report_lines = []
report_lines.append('# Test Results Analysis')
report_lines.append('')
report_lines.append('## Summary Metrics')
for k, v in metrics.items():
    report_lines.append(f'- {k}: {v}')

# attach example figure
last_fig = figs_dir / 'last_plot.png'
if last_fig.exists():
    report_lines.append('\n## Figures')
    report_lines.append(f'![]({last_fig.name})')

report_md.write_text('\n'.join(report_lines), encoding='utf-8')
print('Wrote report to', report_md)


## Section 8: Unit tests (pytest)

Create tests under `tests/` for parsing, cleaning, and metric functions. Example test snippet:

```python
# tests/test_analysis.py
import pandas as pd
from analyze_test_results import clean_unified, compute_metrics

def test_clean_unified_basic():
    df = pd.DataFrame({'status': ['success', 'failure'], 'response_time': ['0.1', '0.2']})
    out = clean_unified(df)
    assert 'response_time' in out.columns

def test_compute_metrics_counts():
    df = pd.DataFrame({'status': ['success','failure','success']})
    m = compute_metrics(df)
    assert m['total'] == 3
    assert m['passes'] == 2
```

Run tests with:

```bash
pytest -q
```


## Section 9: Execute notebook programmatically and capture outputs

You can run this notebook non-interactively with nbconvert or papermill. Examples:

```bash
# Run in-place with nbconvert
jupyter nbconvert --to notebook --execute analyze_test_results.ipynb --output analyze_test_results.executed.ipynb

# Using papermill (parameterize if needed)
papermill analyze_test_results.ipynb analyze_test_results.executed.ipynb
```

Capture stderr/stdout to files when running the commands in a shell.


In [ ]:
# ...existing code...
{
# --- DB接続と可視化セルの追加 ---
# 追加セル: MySQLからデータを読み込み、3つのグラフを描画します。
"""
注意: 下の connection string を実環境の user/password/host/port に書き換えてください。
"""
}
# ...existing code...
{
# 新規セル (Python)
"!pip install pymysql sqlalchemy seaborn --quiet"

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
import time

# DB接続設定（環境に合わせて書き換え）
DB_USER = "root"
DB_PASS = "password"
DB_HOST = "127.0.0.1"
DB_PORT = 3306
DB_NAME = "cloudmart_saga"
CONN_STR = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

engine = sqlalchemy.create_engine(CONN_STR)

# --- クエリ: E2E レイテンシ（ms） ---
q_e2e = """
SELECT order_id,
       created_at,
       COALESCE(confirmed_at, delivered_at, cancelled_at) AS finished_at,
       TIMESTAMPDIFF(MILLISECOND, created_at, COALESCE(confirmed_at, delivered_at, cancelled_at)) AS e2e_ms
FROM orders
WHERE created_at IS NOT NULL
  AND COALESCE(confirmed_at, delivered_at, cancelled_at) IS NOT NULL;
"""
df_e2e = pd.read_sql_query(q_e2e, engine, parse_dates=['created_at','finished_at'])

# --- クエリ: 収束時間（events）秒 ---
q_conv = """
SELECT aggregate_id,
       MIN(processed_at) AS first_at,
       MAX(processed_at) AS last_at,
       TIMESTAMPDIFF(SECOND, MIN(processed_at), MAX(processed_at)) AS convergence_s,
       COUNT(*) AS event_count
FROM events
GROUP BY aggregate_id;
"""
df_conv = pd.read_sql_query(q_conv, engine, parse_dates=['first_at','last_at'])

# --- クエリ: サガのステップタイムライン（補償の可視化） ---
q_saga_steps = """
SELECT sl.saga_id, sl.step_number, sl.step_name, sl.service_name, sl.status,
       sl.started_at, sl.completed_at, si.status AS saga_status
FROM saga_step_logs sl
LEFT JOIN saga_instances si ON sl.saga_id = si.saga_id
ORDER BY sl.saga_id, sl.step_number;
"""
df_steps = pd.read_sql_query(q_saga_steps, engine, parse_dates=['started_at','completed_at'])

# --- 1) E2E レイテンシ: p50/p95/p99 と箱ひげ／ヒストグラム ---
p50 = df_e2e['e2e_ms'].quantile(0.5)
p95 = df_e2e['e2e_ms'].quantile(0.95)
p99 = df_e2e['e2e_ms'].quantile(0.99)

print(f"E2E latency (ms) p50={p50:.1f}, p95={p95:.1f}, p99={p99:.1f}")

plt.figure(figsize=(10,4))
sns.boxplot(x=df_e2e['e2e_ms'])
plt.title('E2E Latency (ms) - boxplot')
plt.show()

plt.figure(figsize=(10,4))
sns.histplot(df_e2e['e2e_ms'], bins=30, kde=False)
plt.axvline(p50, color='C1', linestyle='--', label=f'p50={p50:.0f}ms')
plt.axvline(p95, color='C2', linestyle='--', label=f'p95={p95:.0f}ms')
plt.legend()
plt.title('E2E Latency Histogram (ms)')
plt.xlabel('latency (ms)')
plt.show()

# --- 2) 収束時間ヒストグラム（seconds） ---
plt.figure(figsize=(8,4))
sns.histplot(df_conv['convergence_s'], bins=[0,1,5,30,300,3600], discrete=False)
plt.title('Convergence Time Histogram (s)')
plt.xlabel('seconds to converge')
plt.show()

# 確認用の要約
print(df_conv[['aggregate_id','convergence_s','event_count']].sort_values('convergence_s', ascending=False).head())

# --- 3) 補償／サガタイムライン（Gantt風） ---
# prepare times relative to min time for plotting
if not df_steps.empty:
    df_steps['start_ts'] = df_steps['started_at'].astype('int64') // 10**9
    df_steps['end_ts'] = df_steps['completed_at'].astype('int64') // 10**9
    # fallback for null completed_at
    df_steps['end_ts'] = df_steps['end_ts'].fillna(df_steps['start_ts'] + 1)
    base = df_steps['start_ts'].min()
    df_steps['start_rel'] = df_steps['start_ts'] - base
    df_steps['dur_s'] = df_steps['end_ts'] - df_steps['start_ts']

    # one subplot per saga (or group a few if many)
    sagas = df_steps['saga_id'].unique()
    fig_h = max(4, len(sagas)*1.2)
    fig, ax = plt.subplots(figsize=(12, fig_h))
    y_map = {s: i for i, s in enumerate(sagas[::-1])}
    colors = {'COMPLETED':'C0', 'FAILED':'C3', 'COMPENSATED':'C1', 'PENDING':'C4'}
    for _, row in df_steps.iterrows():
        y = y_map[row['saga_id']]
        c = colors.get(row['status'], 'C7')
        ax.barh(y, row['dur_s'], left=row['start_rel'], height=0.6, color=c, alpha=0.8)
        ax.text(row['start_rel'] + 0.1, y, f"{row['step_name']} ({row['service_name']})", va='center', fontsize=8)

    ax.set_yticks(list(y_map.values()))
    ax.set_yticklabels(list(y_map.keys()))
    ax.set_xlabel('seconds since first recorded step')
    ax.set_title('Saga step timeline (bars = step duration) - color by step status')
    plt.tight_layout()
    plt.show()
else:
    print("No saga step logs found for timeline plot.")
# ...existing code...
}